# Import / Config

In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
from dotenv import load_dotenv

import edurel.utils.rel_schema_man as rsm
import edurel.utils.duckdb as ddb

load_dotenv() 
BASE_DIR = os.getenv("BASE_DIR")
DB_DIR = f"{BASE_DIR}/databases"
CONFIG_DIR = f"{BASE_DIR}/config"


# Database

In [3]:
db = ddb.Db.mem()
db.config_exe(f"{CONFIG_DIR}/databases/company_en.yaml")
schema = db.yaml()
sman = rsm.RelSchemaMan(schema)
sman.add_fks(f"{CONFIG_DIR}/fk_specs/company_en.yaml")
sman.remove_tags(f"{CONFIG_DIR}/omit_tags_specs/dbstuff.yaml")

In [5]:
sman.transform(f"{CONFIG_DIR}/transform_specs/test1.yaml")

In [6]:
print(sman.yaml())

tables:
- tablename: Employee
  columns:
  - columnname: EID
    type: INTEGER
  - columnname: OUID
    type: INTEGER
  - columnname: LastName
    type: VARCHAR
  - columnname: Hiredate
    type: DATE
  - columnname: Salary
    type: DECIMAL(9,2)
  - columnname: Bonus
    type: DECIMAL(9,2)
  primary_key:
  - EID
  foreign_keys:
  - sourcecolumns:
    - OUID
    targettable: OrgUnit
    targetcolumns:
    - OUID
- tablename: OrgUnit
  columns:
  - columnname: OUID
    type: INTEGER
  - columnname: Head
    type: INTEGER
  - columnname: SuperUnit
    type: INTEGER
  - columnname: Name
    type: VARCHAR
  primary_key:
  - OUID
  foreign_keys:
  - sourcecolumns:
    - Head
    targettable: Employee
    targetcolumns:
    - EID
  - sourcecolumns:
    - SuperUnit
    targettable: OrgUnit
    targetcolumns:
    - OEID



# Spec

In [5]:
pec = """
del table pattern: EmpProj
del column OrgUnit index: [1:]
"""

In [ ]:
spec1 = """
del table pattern: EmpProj
del table pattern: Project
"""

spec2 = """
del table pattern: Employee
"""

spec3 = """
del table pattern: Project
"""

spec4 = """
OrgUnit|Head|-->|Employee|EID
OrgUnit|SuperUnit|-->|OrgUnit|OEID
"""

# SMan

In [17]:
sman = smu.SMan(schema)

In [21]:
# sman.reduce(spec1)
# sman.remove_tags(["primary_key", "foreign_keys"])
sman.add_fks(spec4)
print(sman.yaml())

tables:
- tablename: Employee
  columns:
  - columnname: EID
    type: INTEGER
    nullable: false
  - columnname: OUID
    type: INTEGER
    nullable: false
  - columnname: LastName
    type: VARCHAR
    nullable: false
  - columnname: Hiredate
    type: DATE
    nullable: false
  - columnname: Salary
    type: DECIMAL(9,2)
    nullable: false
  - columnname: Bonus
    type: DECIMAL(9,2)
    nullable: true
  primary_key:
  - EID
  foreign_keys:
  - fkname: fk_Employee_OrgUnit_OUID_1
    sourcecolumns:
    - OUID
    targettable: OrgUnit
    targetcolumns:
    - OUID
- tablename: OrgUnit
  columns:
  - columnname: OUID
    type: INTEGER
    nullable: false
  - columnname: Head
    type: INTEGER
    nullable: true
  - columnname: SuperUnit
    type: INTEGER
    nullable: true
  - columnname: Name
    type: VARCHAR
    nullable: false
  primary_key:
  - OUID
  foreign_keys:
  - fkname: fk_OrgUnit_Employee_Head_1
    sourcecolumns:
    - Head
    targettable: Employee
    targetcolumns:
 

In [23]:
sman.save_mermaid_png("test.png")

In [ ]:
fk_spec = {
    "DimAccount": ["DimAccount|ParentAccountKey|AccountKey"],
    "DimDepartmentGroup": ["DimDepartmentGroup|ParentDepartmentGroupKey|DepartmentGroupKey"],
    "DimEmployee": ["DimEmployee|ParentEmployeeKey|EmployeeKey"],
    "DimOrganization": ["DimOrganization|ParentOrganizationKey|OrganizationKey"],
    }

additional_fks = {"Orgunit": ["Employee|Head|EID", "OrgUnit|SuperUnit|OEID"]}


In [13]:
print(sman._fk_spec_to_dict(fk_spec["DimAccount"][0]))

{'DimAccount': [['ParentAccountKey'], ['AccountKey']]}


In [15]:
print(sman.yaml_base())


tables:
- tablename: EmpProj
  columns:
  - columnname: EID
    type: INTEGER
  - columnname: PID
    type: INTEGER
  - columnname: NoOfHoursPerWeek
    type: INTEGER
  primary_key:
  - EID
  - PID
  foreign_keys:
  - sourcecolumns:
    - EID
    targettable: Employee
    targetcolumns:
    - EID
  - sourcecolumns:
    - PID
    targettable: Project
    targetcolumns:
    - PID
- tablename: Employee
  columns:
  - columnname: EID
    type: INTEGER
  - columnname: OUID
    type: INTEGER
  - columnname: LastName
    type: VARCHAR
  - columnname: Hiredate
    type: DATE
  - columnname: Salary
    type: DECIMAL(9,2)
  - columnname: Bonus
    type: DECIMAL(9,2)
  primary_key:
  - EID
  foreign_keys:
  - sourcecolumns:
    - OUID
    targettable: OrgUnit
    targetcolumns:
    - OUID
- tablename: OrgUnit
  columns:
  - columnname: OUID
    type: INTEGER
  - columnname: Head
    type: INTEGER
  - columnname: SuperUnit
    type: INTEGER
  - columnname: Name
    type: VARCHAR
  primary_key:
  -